In [1]:
import numpy as np
import os
import pcl
import rosbag
import rospy
import tf2_ros as tf

from open3d import *
from geometry_msgs.msg import TransformStamped
#from sensor_msgs import point_cloud2 as pc2
from sensor_msgs.msg import Image, PointCloud, PointCloud2
from __future__ import print_function

/usr/lib/python2.7/dist-packages/rosdep2/platforms/pip.py:32: UserWarning: Module pcl was already imported from /usr/local/lib/python2.7/dist-packages/python_pcl-0.3-py2.7-linux-x86_64.egg/pcl/__init__.pyc, but /home/neil/fetch/svm is being added to sys.path
  import pkg_resources


## load data from bags

In [2]:
def bagfile_to_pcl(path, topic='head_camera/depth/points', frame_limit=1):
    bag, points = rosbag.Bag(path), np.zeros(shape=(1,))
    for i, (_, msg, t) in enumerate(bag.read_messages(topics=[topic])):
        if i>frame_limit:
            break
        print(msg) # TODO finish this
    bag.close()
    return pcl.PointCloud(np.array(points, dtype=np.float32))

In [3]:
#targets = ['bolts-on-table', 'single-bolt', 'bolts-in-bin']
targets = ['1556416031.657706491.pcd']
datasets = []

if not all([target in os.listdir('../data/pcd') for target in targets]):
    raise Exception('Missing data.')

### pcd analysis
Voxellize down to $0.5 cm^3$

In [4]:
import ipyvolume as ipv

def view(X):
    if type(X)!='numpy.ndarray':
        N = np.asarray(X).T
    return ipv.quickscatter(N[0,], N[1,], N[2,], size=1, marker='point_2d')

In [5]:
def transform(X, rads=0, axis='z'):
    R = np.array([[1, 0, 0], [0, np.cos(rads), -np.sin(rads)], [0, np.sin(rads), np.cos(rads)]])
    return np.matmul(np.asarray(X), R)

In [6]:
X = pcl.load('../data/pcd/' + targets[0])

In [7]:
view(X)

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


### transform to base frame

In [8]:
import tf

`bag` takes a long time to load, avoid rerunning this.
(pickle?)

In [9]:
bag = rosbag.Bag('../data/single-bolt.bag')
bag.get_message_count()

1615

In [12]:
points = []
for (_, msg, t) in bag.read_messages(topics=['head_camera/depth/points']):
    points.append(msg)

In [18]:
rospy.init_node('svm')
listener = tf.TransformListener()
while not rospy.Time.now():
    pass

In [19]:
(trans, rot) = listener.lookupTransform('head_camera_depth_frame', 'base_link', rospy.Time(0))

In [21]:
listener.transformPointCloud(point_cloud=points[0], target_frame='base_link')

AttributeError: '_sensor_msgs__PointCloud2' object has no attribute 'channels'

In [37]:
X[100000]

(-0.24851146340370178, -0.1231483668088913, 0.812000036239624)

In [28]:
N = np.asarray(X)
N.shape

(307200, 3)

In [30]:
m = TransformStamped()
m.header.frame_id = "head_camera_optical_frame"
br.setTransform(m)

AttributeError: TransformBroadcaster instance has no attribute 'setTransform'

In [ ]:
X = X.make_passthrough_filter()
X.set_filter_field_name('z')